<a href="https://colab.research.google.com/github/Thando367/Prediction-of-Product-Sales/blob/main/Project_1_Part_5(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Project 1 - Part 5 (Core)

<h5>Thandolwethu Siloyi

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# Import packages
import pandas as pd
import numpy as np

In [34]:
#import to read data
df_sales = pd.read_csv('/content/drive/MyDrive/StudentFolders16wk20wk/CodingDojo/02-MachineLearning/Week06/Data/sales_predictions_2023.csv')
df_sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [35]:
df2 = df_sales.copy()

In [36]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [37]:
# checking duplicates
df_sales.duplicated().sum()
# no duplicates

0

In [38]:
# checking datatypes
df_sales.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [39]:
# fix inconsistencies in categorical data.
# Check unique values in each categorical column
string_cols = df_sales.select_dtypes('object').columns

for col in string_cols:
  print(f'Unique values for {col}')
  print(df_sales[col].value_counts())
  print('\n')


Unique values for Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64


Unique values for Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64


Unique values for Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64


Unique values for Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    

In [40]:
# checking inconsistancies for numerical
num_cols = df_sales.select_dtypes('number').columns

for number in num_cols:
  print(f'Unique values for{number}')
  print(df_sales[number].value_counts())
  print('\n')

Unique values forItem_Weight
12.150    86
17.600    82
13.650    77
11.800    76
15.100    68
          ..
7.275      2
7.685      1
9.420      1
6.520      1
5.400      1
Name: Item_Weight, Length: 415, dtype: int64


Unique values forItem_Visibility
0.000000    526
0.076975      3
0.162462      2
0.076841      2
0.073562      2
           ... 
0.013957      1
0.110460      1
0.124646      1
0.054142      1
0.044878      1
Name: Item_Visibility, Length: 7880, dtype: int64


Unique values forItem_MRP
172.0422    7
170.5422    6
196.5084    6
188.1872    6
142.0154    6
           ..
97.3384     1
83.1934     1
96.6752     1
152.6682    1
75.4670     1
Name: Item_MRP, Length: 5938, dtype: int64


Unique values forOutlet_Establishment_Year
1985    1463
1987     932
1999     930
1997     930
2004     930
2002     929
2009     928
2007     926
1998     555
Name: Outlet_Establishment_Year, dtype: int64


Unique values forItem_Outlet_Sales
958.7520     17
1342.2528    16
703.0848     15
1845

In [41]:
# fixing inconsistancies
# for Item_Fat_Content
def fat_content(value):
    value = value.lower()
    if 'low fat' in value:
        return 'Low Fat'
    elif 'reg' in value:
        return 'Regular'
    elif 'lf' in value:
        return 'Low Fat'
    else:
        return value

df_sales['Item_Fat_Content'] = df_sales['Item_Fat_Content'].apply(fat_content)



In [42]:
# fixing inconsistancies for Outlet_Type
def outlet_type(value):
  value = value.lower()
  if 'supermarket type1' in value:
    return 'Supermarket Type1'
  elif 'supermarket type2' in value:
    return 'Supermarket Type2'
  elif 'supermarket type3' in value:
    return 'Supermarket Type3'
  elif 'grocery store' in value:
    return 'Grocery Store'
  else:
        return value

# applying function
df_sales['Outlet_Type'] = df_sales['Outlet_Type'].apply(outlet_type)
df_sales

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


<h1> Identify the features (X) and target (y)

In [43]:
# new imports
# Import packages
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)
import missingno
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector

# Set pandas as the default output for sklearn
from sklearn import set_config
set_config(display='diagram')


In [44]:
# define feature X and exclude name column
target = 'Item_Outlet_Sales'
X = df_sales.drop(columns=[target]).copy()

# define target y as Item_Outlet_Sales
y = df_sales[target].copy()

In [45]:
# performing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [46]:
X_train.shape

(6392, 11)

In [47]:
X_test.shape

(2131, 11)

In [48]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

<h1>Create a preprocessing object to prepare the dataset for Machine Learning

*   Numeric
*   Categorical



<h3> 1. Numerical

In [49]:
# creating a numerical data selector
num_cols = make_column_selector(dtype_include='number')
num_cols(X_train)

['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year']

In [50]:
# to scale the data we have to create the scale
miss_imputer = SimpleImputer(strategy='constant', fill_value='missing')
scaler = StandardScaler()

In [51]:
# Pipeline
num_pipe = make_pipeline(miss_imputer,scaler)
num_pipe

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('standardscaler', StandardScaler())])

In [52]:
# Tuple
#num_tuple = ('numeric', scaler, num_cols)

In [53]:
##############################################################################################################################################################################

<h4> 2. Ordinal

In [54]:
from pandas.core.dtypes.dtypes import Ordered

# Ordinal - Preprocessing Pipeline
ord_cols = ['Item_Fat_Content','Outlet_Location_Type', 'Outlet_Size']

# ordered category lists
fat_list = ['Low Fat', 'Regular']
location_list = ['Tier 1', 'Tier 2', 'Tier 3']
size_list = ['Small', 'Medium', 'High']

# Transformers
order = OrdinalEncoder(categories=[fat_list, location_list, size_list])

impute_cat = SimpleImputer(strategy='constant', fill_value='missing')

# scale
scaler2 = StandardScaler()

# Pipeline
order_pipeline = make_pipeline(impute_cat, order, scaler2)


# Tuple
#ord_tuple = ('ordinal', order_pipeline,ord_cols)
order_pipeline

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['Low Fat', 'Regular'],
                                            ['Tier 1', 'Tier 2', 'Tier 3'],
                                            ['Small', 'Medium', 'High']])),
                ('standardscaler', StandardScaler())])

In [55]:
###############################################################################################################

<h4> 3. Nominal

In [56]:
# Prepeocessing Pipeline
nom_cols = X_train.select_dtypes('object').drop(columns=ord_cols).columns

# Transformers
cat_encoder = SimpleImputer(strategy='constant', fill_value='missing')

# scale
scaler3 = StandardScaler()

# Pipeline
nom_pipe = make_pipeline(cat_encoder, scaler3)

# Tuple
#nom_tuple = ('categorical', nom_pipe, nom_cols)
nom_pipe

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('standardscaler', StandardScaler())])

In [57]:
nom_cols

Index(['Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Outlet_Type'], dtype='object')

In [66]:
#Create a tuple for each transformer
num_tuple = ('Numeric', scaler, num_cols)
ord_tuple = ('Ordinal', impute_cat, ord_cols)
nom_tuple = ('Categorical', cat_encoder, nom_cols)

In [67]:
# making a column transformer
column_trans = ColumnTransformer([num_tuple, ord_tuple, nom_tuple], remainder='drop', verbose_feature_names_out=False)


In [68]:
#Fit column transformer
column_trans.fit(X_train, y_train)

ColumnTransformer(transformers=[('Numeric', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7b2a54a4f790>),
                                ('Ordinal',
                                 SimpleImputer(fill_value='missing',
                                               strategy='constant'),
                                 ['Item_Fat_Content', 'Outlet_Location_Type',
                                  'Outlet_Size']),
                                ('Categorical',
                                 SimpleImputer(fill_value='missing',
                                               strategy='constant'),
                                 Index(['Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Outlet_Type'], dtype='object'))],
                  verbose_feature_names_out=False)